In [169]:
import numpy as np
import geopandas as gpd
import pandas as pd
import os

In [170]:
#Set File path holding shapefiles
file = os.listdir("data/shapefile")
#set path for all files with .shp file type
path = [os.path.join("data/shapefile", i) for i in file if ".shp" in i]

#Create new geodata drame with concatenated shapefiles.  Merging all files into 1 geodata frame
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in path], 
                        ignore_index=True), crs=gpd.read_file(path[0]).crs)



In [171]:
#filter geodata frame byt vegeation type
turf = gdf[gdf['type']=='turf']

#dissolve polygons
#turf_dissolve = turf.dissolve(by='type')
turf_CO83CF = turf.to_crs("EPSG:2232")

In [172]:
#load parcel polygons and convert to WGS84
parcels = gpd.read_file('data/parcels.shp')
parcels_CO83CF  = parcels.to_crs("EPSG:2232")

In [173]:
clipped = gpd.clip(parcels_CO83CF, turf_CO83CF)

In [174]:
clipped['area']=clipped.geometry.area

In [175]:
clipped.head()

,address,id,geometry,area
0,302 CASTLE PINES DR SOUTH,1,"MULTIPOLYGON (((3173294.000 1585004.500, 31732...",1279.858929
1,391 STARMINE PL,2,"MULTIPOLYGON (((3173889.000 1585016.000, 31738...",563.223228
2,4537 SILVER DALE CT,3,"MULTIPOLYGON (((3173439.000 1582344.500, 31734...",883.085604
4,4 BOREALIS WAY,5,"MULTIPOLYGON (((3172336.438 1586847.500, 31723...",802.323934
5,655 RUBY TRUST DR,6,"MULTIPOLYGON (((3173030.500 1580174.500, 31730...",2868.543115


In [176]:
dis_clip = clipped.dissolve(by='id', aggfunc='sum')

In [177]:
dis_clip.head()

,geometry,area
id,,
1,"MULTIPOLYGON (((3173293.596 1585004.500, 31732...",1279.858929
2,"MULTIPOLYGON (((3173890.000 1585017.000, 31738...",563.223228
3,"MULTIPOLYGON (((3173438.000 1582345.500, 31734...",883.085604
5,"MULTIPOLYGON (((3172337.500 1586849.000, 31723...",802.323934
6,"MULTIPOLYGON (((3173032.000 1580174.500, 31730...",2868.543115


In [178]:
dis_clip.to_file("data/turf_polygons.json", driver="GeoJSON")

In [179]:
dis_clip.drop(columns=['geometry'], inplace=True)
parcel_join = parcels_CO83CF.merge(dis_clip, on='id', how='left')

In [180]:
parcel_join.head()

,address,id,geometry,area
0,302 CASTLE PINES DR SOUTH,1,"POLYGON ((3173554.294 1584793.537, 3173341.527...",1279.858929
1,391 STARMINE PL,2,"POLYGON ((3174051.587 1584979.303, 3173746.736...",563.223228
2,4537 SILVER DALE CT,3,"POLYGON ((3173535.346 1582334.968, 3173517.596...",883.085604
3,3128 RAMSHORN DR,4,"POLYGON ((3166418.574 1585769.111, 3166418.074...",NaN
4,4 BOREALIS WAY,5,"POLYGON ((3172722.590 1586854.849, 3172676.340...",802.323934


In [185]:
parcel_join = parcel_join.to_crs("EPSG:4326")
parcel_join.to_file("data/parcels_with_turf_area.json", driver="GeoJSON")